In [18]:
import sys
sys.path.append('..')

import os
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
from geopandas.tools import sjoin

from utilities_amigocloud import AmigocloudFunctions

In [19]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\catastro_S09_MIERCOLES.shp'
PATH_XLSX_GRUPOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'

In [20]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_pulverizacion.planificacion_pulv
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.planificacion_pulv
                SET procesado = true
                WHERE id = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def obtener_parte_diario_por_os(ors):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv where os = {ors}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    gdf_pla_gral['mezcla'] = gdf_pla_gral['mezcla'].astype(int)
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 35248
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_360912 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, mezcla, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['os']}', '{row['mezcla']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
        """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f'Lote registrado en AmigoCloud')
    return None

def get_catastro():
    engine = obtener_engine()
    try:
        query = f'''
            SELECT * FROM drones_control_bio.catastro_drones
        '''
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener la capa de catastro: {e}")
        return gpd.GeoDataFrame()
    return None

In [21]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [24]:
gdf_cat = get_catastro()
gdf_plan = obtener_planificacion()
gdf_plan

,id,geom,codigo_canero,nombre_canero,procesado,mezcla,obs
0,63,"MULTIPOINT (478275.190 8093944.100, 478519.357...",1530,C.I.T.T.C.A.,False,1,None
1,64,"MULTIPOINT (478325.392 8094270.416, 478380.159...",1530,C.I.T.T.C.A.,False,2,None
2,65,"MULTIPOINT (479085.277 8094021.685, 479092.123...",1530,C.I.T.T.C.A.,False,3,None


# CARGAR PARTE DIARIO

In [29]:
gdf_intersect_result = gpd.sjoin(gdf_cat, gdf_plan, how='inner')
os_para_cargar = list(set(gdf_intersect_result['id_right']))
os_para_cargar

[64, 65, 63]

In [30]:
gdf_intersect_result['mezcla']

9933     1
12935    3
12938    2
12939    3
12940    1
12941    2
12942    3
12943    2
12944    2
12945    1
12946    1
12947    1
12948    1
12950    1
12951    1
12952    3
12953    3
12954    1
12955    1
12956    1
12957    1
12958    1
12959    1
12960    1
12961    1
12962    1
12964    1
12970    3
12971    3
12972    3
12973    3
12974    3
12975    3
12977    3
12978    3
12980    3
12981    3
12982    3
12983    3
12984    3
12985    1
12987    1
12988    1
12990    1
12993    1
12994    1
12995    1
12996    1
12997    1
12998    3
12999    2
13000    2
13001    2
13002    1
13634    2
Name: mezcla, dtype: int64

In [31]:
gdf_result = gdf_intersect_result[['geom', 'unidad_01', 'unidad_02', 'codigo_canero', 'nombre_canero', 'unidad_05', 'area', 'id_right', 'soca', 'zona', 'mezcla']].copy()
gdf_result['area'] = gdf_result.geometry.area / 10000
gdf_result.rename(columns={
    'geometry': 'geom',
    'codigo_canero': 'unidad_03',
    'nombre_canero': 'unidad_04',
    'id_right': 'os',
    'zona': 'inst'
}, inplace=True)

gdf_result = gdf_result.set_geometry("geom")
gdf_result['unidad_01'] = gdf_result['unidad_01'].astype(int)
gdf_result['unidad_03'] = gdf_result['unidad_03'].astype(int)
gdf_result['os'] = gdf_result['os'].astype(int)
gdf_result['soca'] = gdf_result['soca'].astype(int)
gdf_result['inst'] = gdf_result['inst'].astype(int)
gdf_result['mezcla'] = gdf_result['mezcla'].astype(int)

In [32]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst,mezcla
9933,"MULTIPOLYGON (((479108.232 8093816.937, 479108...",299,CITTCA,1530,C.I.T.T.C.A.,L14.2,0.560878,63,1,124,1
12935,"MULTIPOLYGON (((479990.326 8092837.357, 479989...",299,CITTCA,1530,C.I.T.T.C.A.,L31,4.077693,65,3,124,3
12938,"MULTIPOLYGON (((480646.958 8092665.233, 480679...",299,CITTCA,1530,C.I.T.T.C.A.,L28.2,3.007359,64,2,124,2
12939,"MULTIPOLYGON (((480241.833 8093166.959, 480241...",299,CITTCA,1530,C.I.T.T.C.A.,L29,2.948671,65,1,124,3
12940,"MULTIPOLYGON (((478713.705 8093836.114, 478713...",299,CITTCA,1530,C.I.T.T.C.A.,L5,6.174679,63,1,124,1
12941,"MULTIPOLYGON (((478719.217 8094334.189, 478711...",299,CITTCA,1530,C.I.T.T.C.A.,L3,11.121721,64,1,124,2
12942,"MULTIPOLYGON (((480591.353 8093305.757, 480507...",299,CITTCA,1530,C.I.T.T.C.A.,L22,8.957423,65,1,124,3
12943,"MULTIPOLYGON (((478458.580 8094202.368, 478453...",299,CITTCA,1530,C.I.T.T.C.A.,L2.2,2.282723,64,4,124,2
12944,"MULTIPOLYGON (((478189.223 8094345.350, 478328...",299,CITTCA,1530,C.I.T.T.C.A.,L2.1,3.216577,64,1,124,2
12945,"MULTIPOLYGON (((478952.792 8094329.269, 478972...",299,CITTCA,1530,C.I.T.T.C.A.,L4.2,5.092228,63,2,124,1


In [33]:
# GARANTIZAR LA INST DEL CAÑERO
# cargar grupos de cosecha
df_grupos = pd.read_excel(PATH_XLSX_GRUPOS, sheet_name='CODIGOS')
df_grupos = df_grupos[df_grupos['INSTITUCION'].notna()]
df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
df_grupos['INSTITUCION'] = df_grupos['INSTITUCION'].astype(int)

# Crear un diccionario de mapeo: {codigo_cañero: institucion}
mapa_institucion = df_grupos.set_index('CODIGO CAÑERO')['INSTITUCION'].to_dict()
# Reemplazar los valores de 'inst' en gdf_result usando el diccionario
# a partir del diccionario, busca el codigo cañero (unidad_03), y reempalza la institucion
gdf_result['inst'] = gdf_result['unidad_03'].map(mapa_institucion)

In [34]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst,mezcla
9933,"MULTIPOLYGON (((479108.232 8093816.937, 479108...",299,CITTCA,1530,C.I.T.T.C.A.,L14.2,0.560878,63,1,124,1
12935,"MULTIPOLYGON (((479990.326 8092837.357, 479989...",299,CITTCA,1530,C.I.T.T.C.A.,L31,4.077693,65,3,124,3
12938,"MULTIPOLYGON (((480646.958 8092665.233, 480679...",299,CITTCA,1530,C.I.T.T.C.A.,L28.2,3.007359,64,2,124,2
12939,"MULTIPOLYGON (((480241.833 8093166.959, 480241...",299,CITTCA,1530,C.I.T.T.C.A.,L29,2.948671,65,1,124,3
12940,"MULTIPOLYGON (((478713.705 8093836.114, 478713...",299,CITTCA,1530,C.I.T.T.C.A.,L5,6.174679,63,1,124,1
12941,"MULTIPOLYGON (((478719.217 8094334.189, 478711...",299,CITTCA,1530,C.I.T.T.C.A.,L3,11.121721,64,1,124,2
12942,"MULTIPOLYGON (((480591.353 8093305.757, 480507...",299,CITTCA,1530,C.I.T.T.C.A.,L22,8.957423,65,1,124,3
12943,"MULTIPOLYGON (((478458.580 8094202.368, 478453...",299,CITTCA,1530,C.I.T.T.C.A.,L2.2,2.282723,64,4,124,2
12944,"MULTIPOLYGON (((478189.223 8094345.350, 478328...",299,CITTCA,1530,C.I.T.T.C.A.,L2.1,3.216577,64,1,124,2
12945,"MULTIPOLYGON (((478952.792 8094329.269, 478972...",299,CITTCA,1530,C.I.T.T.C.A.,L4.2,5.092228,63,2,124,1


In [35]:
# agregar nuevos registros a la base de datos
gdf_result.to_postgis("parte_diario_pulv", obtener_engine(), schema="drones_pulverizacion", if_exists="append")

In [36]:
os_para_cargar

[64, 65, 63]

In [37]:
# CARGAR A AMIGOCLOD
for i in os_para_cargar:
    plan_os = obtener_parte_diario_por_os(i)
    cargar_a_amigocloud(plan_os)
    marcar_como_procesado(i)

Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 64 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 65 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote